In [1]:
import scipy
import numpy as np
import matplotlib.pyplot as plt

from scipy.sparse import csc_matrix, csr_matrix, coo_matrix
from scipy.sparse.linalg import aslinearoperator

np.set_printoptions(linewidth=100000)
np.set_printoptions(precision=5)

from pyiga import bspline, assemble, vform, geometry, vis, solvers, topology, bspline_cy


In [2]:
kv = bspline_cy.KnotVector(2, np.array([0.0,0.0,0.0,0.25,0.5,0.75,1.0,1.0,1.0], dtype=np.float64))

In [3]:
print(kv)

Knot vector (degree 2): [0.000, 0.000, 0.000, 0.250, 0.500, 0.750, 1.000, 1.000, 1.000]


In [4]:
a = kv.greville()

In [11]:
kv = bspline_cy.make_knots(2,0.0,1.0,500, mult=2)

In [ ]:
print(kv)

In [26]:
bspline

<module 'pyiga.bspline' from '/home/wolfman/miniforge3/lib/python3.12/site-packages/pyiga/bspline.py'>